# <div align="center">Detrital Principal Component Analysis</div>
## <div align="center">Principal component analysis of trace element geochemistry data in R</div>
#### <div align="center">Megan Mueller</div>
##### <div align="center">Please cite the accompanying article under consideration in Geochronology:</div>
<div align="center">Mueller et al., https://doi.org/10.5194/egusphere-2023-1293 </div>



<br><div align="center">Version 1.1.3</div>

## Overview 
This notebook performs principal component analysis on trace element data from detrital rutile. 

### Initial Conditions
The code runs an R kernel, so R must be installed on the local computer (https://cran.r-project.org/) and added to the system Path (Anaconda guide: https://www.datacamp.com/tutorial/installing-anaconda-windows; R guide: https://www.hanss.info/sebastian/post/rtools-path/). After R is installed and added to the path, R can be run here in jupyter.

The code was written using file input as an .xlsx file. The code can be modified for other file input structures. 


#### Organization
This notebook is divided into the following sections: 
1. [Data import and initial code](#Section1) (must run first)
2. [PCA and tables](#Section2)
3. [2D biplots](#Section3)

<a id='Section1'></a>
# 1. Import Data and Set-Up


## 1.1 Install R Packages and Import Libraries

In [ ]:
# Install R packages
# Run to see if installed, if not, will install

install_if_not_installed <- function(package_name) {
  if (!requireNamespace(package_name, quietly = TRUE)) {
    install.packages(package_name)
    cat(sprintf("Installed package: %s\n", package_name))
  } else {
    cat(sprintf("Package %s is already installed.\n", package_name))
  }
}

# Install IRkernel
install_if_not_installed("IRkernel")
IRkernel::installspec(user = FALSE)  # This line installs the R kernel for Jupyter
install_if_not_installed("IRdisplay")

# Install readxl
install_if_not_installed("readxl")

# Install openxlsx
install_if_not_installed("openxlsx")

# Install robCompositions
install_if_not_installed("robCompositions")

# Install ggplot2
install_if_not_installed("ggplot2")

# Install pls
install_if_not_installed("pls")

# Install data.table
install_if_not_installed("data.table")

# Install cvTools
install_if_not_installed("cvTools")


## 1.2 Load geochemical data
The file must include columns for each element composition (ppm) of interest. Update the folder_path, file_name, and sheet_name of the Excel file

In [ ]:
# Load geochemical data

# load package
library(readxl)

# Edit the following lines to direct to the Excel file with trace element data
folder_path1 <- 'C:/Users/megan/Dropbox/CommonPbPython/Example-data'  # Update this with the actual path
file_name1 <- 'ExampleData-RutileTraceElements-Muelleretal.xlsx'  # Update this with the actual first file name
sheet_name1 <- 'Sheet1'  # Sheet name in Excel file

excel_file_path1 <- file.path(folder_path1, file_name1)  # Combine folder path and file name using file.path
df1 <- readxl::read_excel(excel_file_path1, sheet = sheet_name1)  # Load the first file into a data frame

# Check for and resolve duplicated column names
if (any(duplicated(names(df1)))) {
  warnings("Duplicate column names found. Attempting to resolve.")
  for (i in seq_along(names(df1))) {
    if (any(names(df1) == names(df1)[i])) {
      names(df1)[i] <- paste0(names(df1)[i], "_", i)
    }
  }
}

View(df1)

Load geochemical data from imported spreadsheet. Load variables by header names. Update *geochem_data* to include all elements of interest.

In [ ]:
# Select which columns to include in the PCA
# Replace with the actual names of the columns you want to select
V <- df1[, "V"]
Cr <- df1[, "Cr"]
Zn <- df1[, "Zn"]
Ga <- df1[, "Ga"]
Zr <- df1[, "Zr"]
Nb <- df1[, "Nb"]
Mo <- df1[, "Mo"]
Sn <- df1[, "Sn"]
Sb <- df1[, "Sb"]
Hf <- df1[, "Hf"]
Ta <- df1[, "Ta"]
W <- df1[, "W"]

# # Combine the vectors into a new data frame or matrix, include all elements of interest
#geochem_data <- data.frame(V, Cr, Zn, Ga, Zr, Nb, Mo, Sn, Sb, Hf, Ta, W)
geochem_data <- data.frame(V, Cr, Zn, Zr, Nb, Hf, Ta, W)


# Decide how to handle cells with 0.0 or NaN values
geochem_data[geochem_data <= 0] <- 1e-6 # convert 0.0 or negative values to very small number
#geochem_data[geochem_data <= 0] <- NA # convert 0.0 or negative values to NaN then omit entire row (analysis) in next line (likely biases results)

# omit NaN values by omitting entire row (analysis) if cell has NaN
geochem_data <- na.omit(geochem_data) 

# print geochem_data summary and table
print("geochem_data summary:")
summary(geochem_data)
print("geochem_data:")
View(geochem_data)


# 2. Perform PCA on Compositional Data
The  robCompositions package is specifically designed for Principal Component Analysis (PCA) on compositional data. The pcaCoDa function performs PCA on compositional data and is aware of the constraints imposed by the simplex (the set of compositional data). pcaCoDa uses log-ratio transformations to transform compositional data before applying PCA.

The compositional dataset is expressed in logratio coordinates. Afterwards, robust principal component analysis is performed. Resulting loadings and scores are back-transformed to the clr space where the compositional biplot can be shown.

More on compositional data analysis here: http://www.sediment.uni-goettingen.de/staff/tolosana/extra/CoDaNutshell.pdf

In [ ]:
# more about pcaCoDa here
help(pcaCoDa)

## 2.1 Principal component analysis

In [ ]:
# Perform PCA on compositional data using pcaCoDa from robCompositions package

# load libraries
library(robCompositions)
library(ggplot2)
library(pls)
library(data.table)
library(cvTools)

# Perform robust PCA
pca_result <- pcaCoDa(geochem_data, method = "ILR") # default logratio transformation is isometric (IRL), also additive (ALR), centered (CLR)

# Scores and loadings
scores <- pca_result$scores
loadings <- pca_result$loadings

# Eigenvalues
eigenvalues <- pca_result$eigenvalues

# Calculate percent variance of each principal component
percent_variance <- (eigenvalues / sum(eigenvalues)) * 100

# Calculate cumulative proportion of variance
cumulative_variance <- cumsum(eigenvalues) / sum(eigenvalues)

# Access PCA results
print("PCA results:")
summary(pca_result)

# Outlier detection
#oD <- outCoDa(geochem_data)
#oD
#print("Outliers. Not excluded.")
#plot(oD)

## 2.2 Scree Plot
View PCA results as scree plot. Option to save as PDF.

In [ ]:
# Scree plot
library(ggplot2)
library(gridExtra)

# Set theme options for ggplot
custom_theme <- theme_bw() + theme (text = element_text(size = 16)) + theme(text = element_text(face = "bold"))

# Set up a 2x1 layout for subplots
par(mfrow = c(2, 1))

# Increase size: width = 10, height = 8
options(repr.plot.width = 10, repr.plot.height = 12)

# Top subplot: Percent variance
p1 <- ggplot() +
  geom_line(aes(x = 1:length(eigenvalues), y = percent_variance), color = "black", linewidth = 1.25) +
  geom_point(aes(x = 1:length(eigenvalues), y = percent_variance), color = "black", size = 5) +
  labs(x = "Principal Component", y = "Variance (%)", title = "Percent Variance") +
  scale_x_continuous(breaks = seq(1, length(eigenvalues), by = 1)) +  # Set x-axis increments
  custom_theme

# Bottom subplot: Cumulative variance
p2 <- ggplot() +
  geom_line(aes(x = 1:length(eigenvalues), y = cumulative_variance), color = "black", linewidth = 1.25) +
  geom_point(aes(x = 1:length(eigenvalues), y = cumulative_variance), color = "black", size = 5) +
  labs(x = "Principal Component", y = "Cumulative Proportion of Variance", title = "Cumulative Proportion of Variance") +
  ylim(0, 1) +
  scale_x_continuous(breaks = seq(1, length(eigenvalues), by = 1)) +  # Set x-axis increments
  custom_theme

# Arrange and display the plots
grid.arrange(p1, p2)

# Reset the plotting layout to default
par(mfrow = c(1, 1), oma = c(0, 0, 0, 0))

# Reset size to default
options(repr.plot.width = 7, repr.plot.height = 7)


In [ ]:
# Run to save scree plot
# Specify the folder path
folder_path <- "C:/Users/megan/Desktop"

# Specify the file name
file_name <- "ScreePlot_v1.pdf"

# Combine folder path and file name
pdf_file_path <- file.path(folder_path, file_name)

# Save the ggplot objects to a PDF file
ggsave(pdf_file_path, arrangeGrob(p1, p2), width = 10, height = 12)

cat("Plot successfully saved to", pdf_file_path, "\n")

## 2.3 PCA Results Tables
This section generates various PCA output tables with the final cell saving the tables to an Excel file

In [ ]:
# Correlation matrix

cor_matrix <- cor(geochem_data)

# Display the correlation matrix as a table
print("Correlation Matrix:")
print(cor_matrix)

In [ ]:
# Eigenvalues table

eigenvalue_table <- data.frame(
  "Principal Component" = 1:length(eigenvalues),
  "Eigenvalue" = eigenvalues,
  "Percentage of Variance" = percent_variance,
  "Cumulative Variance" = cumulative_variance
)

# Print the eigenvalue table
print("Eigenvalues of the Correlation Matrix")
print(eigenvalue_table)

In [ ]:
# Eigenvectors table

# Create a data frame for the eigenvectors table
eigenvectors_table <- data.frame(
  "Element" = rownames(loadings),
  apply(loadings, 2, function(x) sprintf("%.4f", x))
)

# Print the eigenvectors table
print("Extracted Eigenvectors")
print(eigenvectors_table)

### Save to Excel file
Run the next cell to save the above tables to one Excel file. Change the folder_path and file_name

In [ ]:
# Run to save above tables to Excel file

# Install and load the required package if not already installed
library(openxlsx)

# Assuming cor_matrix, eigenvalue_table, and eigenvectors_table are your tables

# Set the folder path
folder_path <- "C:/Users/megan/Desktop"

# Define the file name
file_name <- "PCA_results.xlsx"

# Full file path
full_path <- file.path(folder_path, file_name)

# Create a workbook
wb <- createWorkbook()

# Add each table as a separate sheet
addWorksheet(wb, sheetName = "CorrelationMatrix")
writeData(wb, sheet = "CorrelationMatrix", x = cor_matrix, rowNames=TRUE)

addWorksheet(wb, sheetName = "EigenvalueOfCorrelationMatrix")
writeData(wb, sheet = "EigenvalueOfCorrelationMatrix", x = eigenvalue_table, rowNames=TRUE)

addWorksheet(wb, sheetName = "ExtractedEigenvectors")
writeData(wb, sheet = "ExtractedEigenvectors", x = eigenvectors_table, rowNames=TRUE)

# Save the workbook to Excel file
saveWorkbook(wb, full_path)

# Print a completion message
cat("Tables successfully exported to", full_path, "\n")

# 3. PCA Biplots
The next section generates various plots of PCA results

## 3.1 2D biplot with labeled scores and loadings

In [ ]:
# Biplot with score points labeled as observation's index or 
# row number in the dataset used for PCA (df1, geochem_data). Starts at 1.

# Create a biplot
p3 <- biplot(pca_result)

# Specify the folder path
folder_path <- "C:/Users/megan/Desktop"

# Specify the file name
file_name <- "2Dbiplot_ScoresLabeled_v1.pdf"

# Combine folder path and file name
pdf_file_path <- file.path(folder_path, file_name)

# Set a condition (replace this with your actual condition)
save_plot <- TRUE

# Save the ggplot object to a PDF file conditionally
if (save_plot) {
  pdf(pdf_file_path, width = 10, height = 10)
  cat("Plot successfully saved to", pdf_file_path, "\n")
} else {
  cat("Plot not saved.\n")
}

dev.off()


In [ ]:
# Example using base R plot
p3 <- biplot(pca_result)

# Specify the folder path
folder_path <- "C:/Users/megan/Desktop"

# Specify the file name
file_name <- "2Dbiplot_ScoresLabeled_v1.pdf"

# Combine folder path and file name
pdf_file_path <- file.path(folder_path, file_name)

# Set TRUE to save to PDF
save_plot <- FALSE

# Save the base R plot to a PDF file conditionally
if (save_plot) {
  pdf(pdf_file_path, width = 10, height = 12)
  biplot(pca_result)
  dev.off()
  cat("Plot successfully saved to", pdf_file_path, "\n")
} else {
  cat("Plot not saved.\n")
}

## 3.2 2D biplot
Adjust 'scaling_factor' to scale loadings vectors

In [ ]:
# 2D biplot

# Load library
library(ggplot2)
library(IRdisplay)

# Extract scores and loadings from PCA result
scores_2d <- as.data.frame(pca_result$scores[, 1:2])
loadings_2d <- as.data.frame(pca_result$loadings[, 1:2])

# Set a scaling factor for the loadings vectors
scaling_factor <- 6

# Set theme options for ggplot
custom_theme <- theme_bw() + theme (text = element_text(size = 16)) + theme(text = element_text(face = "bold"))

options(repr.plot.width = 10, repr.plot.height = 10)

# Create a scatter plot of scores
p4 <- ggplot(scores_2d, aes(x = Comp.1, y = Comp.2)) +
  geom_point(fill="gray90", size = 4, colour = "black", shape=21, stroke = 1) +
  labs(x = "Principal Component 1", y = "Principal Component 2", title = "PCA Scores") +
  custom_theme

# Add vectors (loadings) to the plot with labels
p4 <- p4 + 
  geom_segment(data = loadings_2d, aes(x = 0, y = 0, xend = scaling_factor * Comp.1, yend = scaling_factor * Comp.2),
                 arrow = arrow(length = unit(0.3, "cm")), color = "darkred", linewidth = 1) +
  geom_text(data = loadings_2d, aes(x = scaling_factor * Comp.1, y = scaling_factor * Comp.2, label = rownames(loadings)),
            hjust = -0.2, vjust = -0.2, color = "darkred", size = 8, fontface = "bold")

print(p4)

In [ ]:
# Run to save 2D biplot plot
# Specify the folder path
folder_path <- "C:/Users/megan/Desktop"

# Specify the file name
file_name <- "2Dbiplot_v1.pdf"

# Combine folder path and file name
pdf_file_path <- file.path(folder_path, file_name)

# Save the ggplot objects to a PDF file
ggsave(pdf_file_path, p4, width = 10, height = 10)

cat("Plot successfully saved to", pdf_file_path, "\n")

## 3.3 2D biplot colored by variable
Set 'color_variable' to color scores by that variable. Modify 'aes(fill=color_variable, size=color_variable)' to set either/both fill color and point size to a variable.

In [ ]:
# 2D biplot with scores fill color and/or size set by a variable: color_variable
library(ggplot2)
library(IRdisplay)

# Color variable
color_variable <-  as.numeric(log(geochem_data$Cr / geochem_data$Nb)) # Cr/Nb example 
#color_variable <- as.numeric(log(geochem_data$Zr)) # Zr example (log)

# Plot theme
custom_theme <- theme_bw() + theme (text = element_text(size = 16)) + theme(text = element_text(face = "bold"))

options(repr.plot.width = 12, repr.plot.height = 8) # width includes legend

# Create a ggplot scatter plot with colored points
p5 <- ggplot(data = NULL, aes(x = scores_2d[, 1], y = scores_2d[, 2])) +
  geom_point(aes(fill=color_variable, size=4), colour="black", shape=21, stroke = 1) +  # set size to color_variable (or other variable) or integer
  labs(x = "PC1", y = "PC2", title = "PCA Scores") +
  scale_fill_continuous(name = "Color Legend: log(Cr/Nb)") +  # Adjust legend title if needed
  custom_theme

print(p5)

In [ ]:
# Run to save 2D biplot plot
# Specify the folder path
folder_path <- "C:/Users/megan/Desktop"

# Specify the file name
file_name <- "2Dbiplot_colorvariable_v1.pdf"

# Combine folder path and file name
pdf_file_path <- file.path(folder_path, file_name)

# Save the ggplot objects to a PDF file
ggsave(pdf_file_path, p5, width = 12, height = 8)

cat("Plot successfully saved to", pdf_file_path, "\n")